In [9]:
import pandas as pd
df = pd.read_excel(r"D:\Proyectos\nllp_antitrust.v1\tunech1ki_dd\data_for_ft.xlsx")

In [10]:
def limpiar_texto(texto):
    texto_limpio = texto.replace('\n', ' ')
    return texto_limpio
df['Clasificación'] = df['Clasificación'].apply(limpiar_texto)
df

,Número de Resolución,Fecha de Resolución,Expediente,Partes,Sumilla,Clasificación,Enlace
0,1-2010/CLC,2010-01-07,010-2009/CLC,Enel S.P.A.,Se declaró fundado el procedimiento iniciado d...,intervención: si materia: otros descripción:...,servicio.indecopi.gob.pe/buscadorResoluciones/...
1,1-2010/ST-CLC,2010-01-27,008-2009/CLC,Pluspetrol Perú Corporation S.A. y Petróleos d...,Se decidió no iniciar procedimiento de oficio ...,intervención: no materia: conductas descripc...,servicio.indecopi.gob.pe/buscadorResoluciones/...
2,8-2010/CLC,2010-01-28,009-2005/CLC,"Juan Carlos Leon Siles, Cosmos Agencia Marítim...",Se declaró improcedente la denuncia interpuest...,intervención: no materia: conductas descripc...,servicio.indecopi.gob.pe/buscadorResoluciones/...
3,9-2010/CLC,2010-02-04,002-2005/CLC,Aero Servicios S.A.C. y Petróleos del Perú – P...,Se declaró infundada la denuncia interpuesta p...,intervención: no materia: conductas descripc...,servicio.indecopi.gob.pe/buscadorResoluciones/...
4,2-2010/ST-CLC,2010-02-23,004-2007/CLC,"Repsol YPF Comercial del Perú S.A., Llama Gas ...",Se decidió no iniciar procedimiento de oficio ...,intervención: no materia: conductas descripc...,servicio.indecopi.gob.pe/buscadorResoluciones/...
...,...,...,...,...,...,...,...
79,52-2023/CLC,2023-05-04,000007-2021/CLC,Cosapi S.A. y el señor Melquiades Damaso Damiá...,Se declaró que Cosapi S.A. y su ejecutivo incu...,intervención: si materia: conductas descripc...,servicio.indecopi.gob.pe/buscadorResoluciones/...
80,55-2023/CLC,2023-05-16,000009-2021/CLC,Electro Dunas S.A.A. y Atria Energía S.A.C.,Se declaró la responsabilidad de Electro Dunas...,intervención: si materia: conductas descripc...,servicio.indecopi.gob.pe/buscadorResoluciones/...
81,62-2023/CLC,2023-06-01,000003-2023/CLC,UNACEM Corp. S.A.A. y Grupo Calidra S.A. de Cv,Se autorizó la operación de concentración empr...,intervención: no materia: concentraciones de...,servicio.indecopi.gob.pe/buscadorResoluciones/...
82,74-2023/CLC,2023-08-04,000006-2023/CLC,FQM Holdings Ltd. y Rio Tinto Western Holdings...,Se autorizó la operación de concentración empr...,intervención: no materia: concentraciones de...,servicio.indecopi.gob.pe/buscadorResoluciones/...


In [11]:
def convert_to_custom_format(dataset):
    converted_data = []
    for _, row in dataset.iterrows():
        json_dict = {
            "messages": [
                {"role": "system", "content": "Vas a clasificar un texto legal. Debes determinar si hubo una intervención indicando 'Si' o 'No'. Luego, debes identificar si corresponde a la materia 'concentraciones', 'conductas' u 'otros'. Luego, debes clasificar si en la categoría de concentraciones hay descripción de 'sin condiciones' o 'condiciones'. Finalmente, debes clasificar si en la descripción de conductas se trata de 'cooperativa' o 'abusiva'."},
                {"role": "user", "content": row['Sumilla']},
                {"role": "assistant", "content": row['Clasificación']}
            ]
        }
        converted_data.append(json_dict)
    return converted_data

converted_data = convert_to_custom_format(df)

In [13]:
from sklearn.model_selection import train_test_split

# Stratified splitting. Assuming 'Top Category' can be used for stratification
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    random_state=42  # for reproducibility
)


In [15]:
import json
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')

training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)

In [18]:
from openai import OpenAI
client = OpenAI(api_key="sk-proj-8vBboP6HGcMcN4tbX4x1T3BlbkFJJfi3FOgF7buaVg8mauzO")

text = 'Autorizar sin condiciones la solicitud de autorización previa de operación de concentración empresarial en el sector eléctrico presentada por ISQ Global Fund GPGP, LTD. y ISQ Global Fund II GP, LLC. para la adquisición del 100% de las acciones de Inkia Americas Ltd. Esta decisión no fue impugnada.'

completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal:competition-law:9Xy3cuWb",
  messages=[
    {"role": "system", "content": "Vas a clasificar un texto legal. Debes determinar si hubo una intervención indicando 'Si' o 'No'. Luego, debes identificar si corresponde a la materia 'concentraciones', 'conductas' u 'otros'. Luego, debes clasificar si en la categoría de concentraciones hay descripción de 'sin condiciones' o 'condiciones'. Finalmente, debes clasificar si en la descripción de conductas se trata de 'cooperativa' o 'abusiva'."},
    {"role": "user", "content": text}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='intervención: no  materia: concentraciones  descripción: sin condiciones', role='assistant', function_call=None, tool_calls=None)
